In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as ny

In [2]:
dataset=pd.read_csv("Social_Network_Ads.csv")

In [3]:
dataset

,User ID,Gender,Age,EstimatedSalary,Purchased
0,15624510,Male,19,19000,0
1,15810944,Male,35,20000,0
2,15668575,Female,26,43000,0
3,15603246,Female,27,57000,0
4,15804002,Male,19,76000,0
...,...,...,...,...,...
395,15691863,Female,46,41000,1
396,15706071,Male,51,23000,1
397,15654296,Female,50,20000,1
398,15755018,Male,36,33000,0


In [4]:
dataset=pd.get_dummies(dataset,drop_first=True)

In [5]:
dataset

,User ID,Age,EstimatedSalary,Purchased,Gender_Male
0,15624510,19,19000,0,True
1,15810944,35,20000,0,True
2,15668575,26,43000,0,False
3,15603246,27,57000,0,False
4,15804002,19,76000,0,True
...,...,...,...,...,...
395,15691863,46,41000,1,False
396,15706071,51,23000,1,True
397,15654296,50,20000,1,False
398,15755018,36,33000,0,True


In [6]:
dataset=dataset.drop("User ID", axis=1)

In [7]:
dataset

,Age,EstimatedSalary,Purchased,Gender_Male
0,19,19000,0,True
1,35,20000,0,True
2,26,43000,0,False
3,27,57000,0,False
4,19,76000,0,True
...,...,...,...,...
395,46,41000,1,False
396,51,23000,1,True
397,50,20000,1,False
398,36,33000,0,True


In [8]:
dataset["Purchased"].value_counts()

Purchased
0    257
1    143
Name: count, dtype: int64

In [9]:
dataset.columns

Index(['Age', 'EstimatedSalary', 'Purchased', 'Gender_Male'], dtype='object')

In [10]:
indep=dataset[['Age', 'EstimatedSalary', 'Gender_Male']]

In [11]:
dep=dataset['Purchased']

In [13]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(indep, dep, test_size=1/3, random_state=0)

In [14]:
indep.shape

(400, 3)

In [15]:
dep

0      0
1      0
2      0
3      0
4      0
      ..
395    1
396    1
397    1
398    0
399    1
Name: Purchased, Length: 400, dtype: int64

In [16]:
from sklearn.preprocessing import StandardScaler
sc=StandardScaler()
x_train=sc.fit_transform(x_train)
x_test=sc.transform(x_test)

In [23]:
from sklearn.linear_model import LogisticRegression

from sklearn.model_selection import GridSearchCV

param_grid={'solver':['newton-cg', 'lbfgs', 'liblinear', 'saga'], 'penalty':['l1', 'l2', 'elasticnet', 'none']}

grid=GridSearchCV(LogisticRegression(), param_grid, refit=True, verbose=3, n_jobs=-1, scoring='f1_weighted')
#to tune a model
grid.fit(x_train, y_train)

Fitting 5 folds for each of 16 candidates, totalling 80 fits


C:\ProgramData\anaconda3\Lib\site-packages\sklearn\model_selection\_validation.py:547: FitFailedWarning: 
50 fits failed out of a total of 80.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
5 fits failed with the following error:
Traceback (most recent call last):
  File "C:\ProgramData\anaconda3\Lib\site-packages\sklearn\model_selection\_validation.py", line 895, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\ProgramData\anaconda3\Lib\site-packages\sklearn\base.py", line 1474, in wrapper
    return fit_method(estimator, *args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\ProgramData\anaconda3\Lib\site-packages\sklearn\linear_model\_logistic.py", line 1172, in fit
    solver = _c

GridSearchCV(estimator=LogisticRegression(), n_jobs=-1,
             param_grid={'penalty': ['l1', 'l2', 'elasticnet', 'none'],
                         'solver': ['newton-cg', 'lbfgs', 'liblinear', 'saga']},
             scoring='f1_weighted', verbose=3)

In [25]:
#to know the best tune parameter
re = grid.cv_results_
#print(re)

grid_prediction=grid.predict(x_test)

from sklearn.metrics import confusion_matrix
cm=confusion_matrix(y_test,grid_prediction)

from sklearn.metrics import classification_report
clf_report=classification_report(y_test,grid_prediction)
print(clf_report)
print(cm)

              precision    recall  f1-score   support

           0       0.89      0.93      0.91        85
           1       0.87      0.80      0.83        49

    accuracy                           0.88       134
   macro avg       0.88      0.86      0.87       134
weighted avg       0.88      0.88      0.88       134

[[79  6]
 [10 39]]


In [27]:
#to know the f1 score
from sklearn.metrics import f1_score
f1_macro=f1_score(y_test, grid_prediction, average='weighted')
print("The f1_macro value of best parameter is {}:".format(grid.best_params_),f1_macro)

The f1_macro value of best parameter is {'penalty': 'l1', 'solver': 'saga'}: 0.8794289739855382


In [28]:
#as a addition to know a receiver operator characteristic area under curve value to identify best classification
from sklearn.metrics import roc_auc_score
roc_auc_score(y_test, grid.predict_proba(x_test)[:,-1])

0.9493397358943578

In [31]:
table = pd.DataFrame.from_dict(re)
table

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_penalty,param_solver,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,0.000913,0.001131,0.000000,0.000000,l1,newton-cg,"{'penalty': 'l1', 'solver': 'newton-cg'}",NaN,NaN,NaN,NaN,NaN,NaN,NaN,7
1,0.001340,0.001061,0.000000,0.000000,l1,lbfgs,"{'penalty': 'l1', 'solver': 'lbfgs'}",NaN,NaN,NaN,NaN,NaN,NaN,NaN,7
2,0.003422,0.001908,0.008140,0.001987,l1,liblinear,"{'penalty': 'l1', 'solver': 'liblinear'}",0.804764,0.811321,0.66593,0.92351,0.922185,0.825542,0.094918,6
3,0.007710,0.003650,0.007369,0.000825,l1,saga,"{'penalty': 'l1', 'solver': 'saga'}",0.804764,0.811321,0.68205,0.92351,0.922185,0.828766,0.089565,1
4,0.004903,0.001922,0.004607,0.001923,l2,newton-cg,"{'penalty': 'l2', 'solver': 'newton-cg'}",0.799620,0.811321,0.68205,0.92351,0.922185,0.827737,0.089864,3
5,0.007951,0.001827,0.005690,0.002058,l2,lbfgs,"{'penalty': 'l2', 'solver': 'lbfgs'}",0.799620,0.811321,0.68205,0.92351,0.922185,0.827737,0.089864,3
6,0.003662,0.001899,0.005102,0.002695,l2,liblinear,"{'penalty': 'l2', 'solver': 'liblinear'}",0.804764,0.811321,0.68205,0.92351,0.922185,0.828766,0.089565,1
7,0.004114,0.001707,0.005609,0.000920,l2,saga,"{'penalty': 'l2', 'solver': 'saga'}",0.799620,0.811321,0.68205,0.92351,0.922185,0.827737,0.089864,3
8,0.001332,0.000804,0.000000,0.000000,elasticnet,newton-cg,"{'penalty': 'elasticnet', 'solver': 'newton-cg'}",NaN,NaN,NaN,NaN,NaN,NaN,NaN,7
9,0.002279,0.001361,0.000000,0.000000,elasticnet,lbfgs,"{'penalty': 'elasticnet', 'solver': 'lbfgs'}",NaN,NaN,NaN,NaN,NaN,NaN,NaN,7


In [32]:
age_input=float(input('Age'))
EstimatedSalary_input=float(input('EstimatedSalary'))
Gender_Male_input=float(input('Gender_Male'))

Age 26
EstimatedSalary 80000
Gender_Male 1


In [33]:
grid.predict([[age_input, EstimatedSalary_input, Gender_Male_input]])

array([1], dtype=int64)